In [1]:
import cv2
import pytesseract
import numpy as np
from pytesseract import Output

In [2]:
def show(images):
    for i,j in enumerate(images):
        images[i] = cv2.resize(j,(500,500))
    
    while True:
        # Press 'q' for exit
        exit_key = ord('q')
        if cv2.waitKey(exit_key) & 255 == exit_key:
            cv2.destroyAllWindows()
            break
        for i,j in enumerate(images):
            name = 'Image {}'.format(i)
            cv2.imshow(name, j)

In [3]:
def generate_mask(image):
    box_image = image.copy()
    inpaint_image = image.copy()

    h, w, c = image.shape
    mask = np.ones(image.shape, dtype=np.uint8)
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = gray

    for i in [6]:
        boxes = pytesseract.image_to_data(thresh,config='--psm {}'.format(i),lang='eng',output_type=Output.DICT)
        for b in zip(boxes['text'],boxes['left'],boxes['top'],boxes['width'],boxes['height'],boxes['conf']):
            if int(b[-1])>20 and len(b[0])>0 and b[0].strip():
                box_image = cv2.rectangle(
                    box_image, (b[1],b[2]), (b[1]+b[3], b[2]+b[4]), (0, 255, 0), 2)
                mask[b[2]:b[2]+b[4],b[1]:b[1]+b[3], :] = 0
    
    for i in range(1):
        inpaint_image = cv2.inpaint(inpaint_image, 1-mask[:, :, 0], 3, cv2.INPAINT_NS)
    
    return inpaint_image,box_image, 255*mask, image*mask

In [4]:
image = cv2.imread('static/original.jpeg')
inpaint_image,box_image, mask, fusion = generate_mask(image)

In [5]:
cv2.imwrite('static/1_box.png', box_image)
cv2.imwrite('static/2_mask.png', mask)
cv2.imwrite('static/3_mask_on_image.png', fusion)
cv2.imwrite('static/4_inpaint.png', inpaint_image)

True